# Boston House Price DataSet

### Eimear Butler G00364802 Machine Learning and Statistics (Dr. Ian McLoughlan)

***

## Introduction

The well known Boston DataSet is routinely used as a teaching aid. It is a natural dataset fist published in 1978 containing US census data concerning houses in various suburbs around the city of Boston. Each of the 506 data entries (suburbs) has 13 characteristics documented. All data entries, with the exception of the column titles, are numerical. 

In this Jupyter Notebook, I aim to 
-  **describe** the DataSet in more detail
-  **analyse** whether there is a significant difference in median house prices between houses that are along the Charles river and those that aren’t
-  create a neural network that can **predict** the median house price based on the other variables in the dataset

In [ ]:
# The packages we will be using are as follows:

import numpy as np      #useful throughout the notebook for analysis of data
import pandas as pd     #useful throughout the notebook for analysis & presentation of data in array form
import scipy.stats as ss #useful throughout the notebook for analysis of data
import matplotlib.pyplot as plt    #useful throughout the notebook for visualisation of data
import seaborn as sns              #useful throughout the notebook for visualisation of data
from tensorflow.python import keras     #useful to develop neural network and make estimations
#print(keras.__version__)
import keras as kr                      #useful to develop neural network and make estimations 
from sklearn import preprocessing as pre       #useful to prepare our data before feeding it into a neural network
from sklearn.metrics import r2_score    #useful to assess accuracy of estimations when working with linear data
import math as math       #for any extra mathematical functions throughout the notebook

In [ ]:
# This just sets the default plot size to be bigger.

plt.rcParams['figure.figsize'] = (20, 10)

In [ ]:
#The Boston Dataset it is available directly from sklearn 
from sklearn.datasets import load_boston          #https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html
boston = load_boston()
print(boston.data.shape)

In [ ]:
#or anaconda which I will use here 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\sklearn\\datasets\\data\\boston_house_prices.csv' 

df0 = pd.read_csv('C:\\ProgramData\\Anaconda3\\lib\\site-packages\\sklearn\\datasets\\data\\boston_house_prices.csv') #or boston['data'] could be used
df1 = pd.DataFrame(df0.values[1:], columns=df0.iloc[0]) # move the 'feature_names' title row to become the Pandas df title   #source: https://stackoverflow.com/questions/26147180/convert-row-to-column-header-for-pandas-dataframe
df = df1.astype(float)
df

In [ ]:
#df.isnull().sum()    # will also check if there are any NaN's in each column, here there are not

In [ ]:
#round(df.describe(), 3) #describe is a useful function to review the overall parameters of yoru dataset

By calling `boston['feature_names'])` we confirm the titles for each of the columns.

Furthermore, using `boston['DESCR']` gives us the description of each feature in each suburb. 

A summary is presented below:


1. **CRIM**    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- per capita crime rate by town 
+ **ZN**       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- proportion of residential land zoned for lots over 25,000 sq.ft.       
+ **INDUS**    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- proportion of non-retail business acres per town
+ **CHAS**     &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)      
+ **NOX**      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- nitric oxides concentration (parts per 10 million)
+ **RM**       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- average number of rooms per dwelling
+ **AGE**      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- proportion of owner-occupied units built prior to 1940       
+ **DIS**      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- weighted distances to five Boston employment centres        
+ **RAD**      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- index of accessibility to radial highways 
+ **TAX**      &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- full-value property-tax rate per \$10,000      
+ **PTRATIO**  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- pupil-teacher ratio by town   
+ **B**        &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- 1000(Bk - 0.63)^2 where Bk is the proportion of [people of African American descent] by town       
+ **LSTAT**    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- % lower status of the population
+ **MEDV**     &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- Median value of owner-occupied homes in $1000's  

## Section 1: The CHAS & MEDV Relationship

Reviewing the data presented above, I can see that the key features we are interested in during the first part of this review are displayed. The MEDV (the mean value of the homes in the suburb) and CHAS (the data documenting which suburbs are by the Charles river) will be particularly relevant to us.


The CHAS is a binary data set meaning that there is only 2 possible entries, '1' if the suburb is by the river and '0'' if it is not. This immediately presents an opportunity for us to split the data into 2 groups to compare. 

In [ ]:
df_chas = df1.iloc[:,3].str.contains('1', regex=False)         #segragate the CHAS entries 
df_chas1 = pd.concat([df['MEDV'], df_chas], axis=1, sort=False)  #add them to the MEDV data column
df_chas1

chas_t = ((df_chas1[df_chas1['CHAS'] == True]).drop(['CHAS'], axis=1)) #identify those that are beside the river
chas_f = (df_chas1[df_chas1['CHAS'] == False]).drop(['CHAS'], axis=1) #and those that are not

print('The number of in scope suburbs by the Charles River are', len(chas_t), 'from 506. This set is now called chas_t')

print('The maximum price from the sample of houses close to the river is $',chas_t['MEDV'].max(),'and minimum $', chas_t['MEDV'].min())
print('The mean price is $', chas_t['MEDV'].mean(), 'while the median is $', chas_t['MEDV'].median())
print('')
print('The number of in scope suburbs NOT by the Charles River are', len(chas_f),  'from 506. This set is now called chas_f')
print('The maximum price from the sample of houses away from the river is $',chas_f['MEDV'].max(),'and minimum $', chas_f['MEDV'].min())
print ('The mean price is $', round(chas_f['MEDV'].mean(),2), 'while the median is $', chas_f['MEDV'].median())

print ('NOTE: the MEDV has clearly been capped at $50k for the purpose of this dataset')

In [ ]:
plt.figure(figsize=(20, 2))

features = ['CHAS'] #https://towardsdatascience.com/linear-regression-on-boston-housing-dataset-f409b7e4a155
target = df['MEDV']

for i, col in enumerate(features):  #create a plot 
    plt.subplot(1, len(features) , i+1)
    x = target
    y = df[col]
    plt.scatter(x, y, marker='o')
    plt.title(col)
    plt.ylabel('CHAS Yes = 1, No = 0')
    plt.yticks(np.arange(0, 1.1, 1.0))
    plt.xlabel('MEDV')

Reviewing the spread of the average prices(MEDV) across the 2 data sets (beside the Charles River (1) or not (0)), confirms that the average price is not very much higher however the min price of 13.4 is geatly influencing the "beside the Charles River" data set. 


Viewing the same data in histograms again shows the difference between the 2 groups.

In [ ]:
sns.distplot(chas_t, color='b', axlabel='MEDV', bins=10) # houses close to the river are in blue
sns.distplot(chas_f, color='r') # houses farther from the river are in red
#sns.distplot(df['MEDV'], color='k')

In [ ]:
ss.ttest_ind(chas_t, chas_f) #source: Machine Learning Lectures, Dr. Ian McLoughlan 2019

The ttest between the 2 groups asks the question, what is the probability that the 2 sample groups are from the same overall population? In this case the p value is extremely low and indicates that the the 2 groups are indeed different. 

In [ ]:
#plt.figure(figsize=(18,8))
correlation_matrix = df.corr().round(2) # Source: https://towardsdatascience.com/linear-regression-on-boston-housing-dataset-f409b7e4a155
sns.heatmap(data=correlation_matrix, annot=True) # annot = True to print the values inside the square

# THe following additional code is added to resolve a known bug while displaying the heatmap source: https://github.com/mwaskom/seaborn/issues/1773
b, t = plt.ylim() # discover the values for bottom and top 
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values
plt.show() 

> "The correlation coefficient ranges from -1 to 1. If the value is close to 1, it means that there  is a strong positive correlation between the two variables. When it is close to -1, the variables have a strong negative correlation." 

Source: https://towardsdatascience.com/linear-regression-on-boston-housing-dataset-f409b7e4a155

As per the above quote, very dark (close to -1) show a strong negative correlation or very light areas (close to 1) a strong positive correlation.  For example, the strongest positive correlation between RAD and TAX is seen with a score of 0.91. This means the suburbs with the most accessibility to radial highways are more likely to pay the full-value property-tax rate per $10,000.

In the case of CHAS and MEDV, a score of 0.18 is assigned indicating that there is no significant correlation between the 2 characteristics. Keeping in mind that only 35 houses of 506 are by the Charles River, I feel this test was worth running if even just to discount that there is no obvious correlation we are overlooking. It could be noted that CHAS does not have any very strong correlations with the other characteristics either and MEDV is in fact the strongest.

This heatmap shows us other interesting correlations too which will move onto next. 

## Section 2 - Broader Analysis of Dataset 

"RM has a strong positive correlation with MEDV (0.7) where as LSTAT has a high negative correlation with MEDV(-0.74)."

As a principle, we shouldnt pick both of these features for machine learning, just one, same with AGE and DIS

‘RM’, ‘LSTAT’(OUT), ‘PTRATIO’ and ‘MEDV’. are the most relevant 

RM                - average number of rooms per dwelling
PTRATIO      - pupil-teacher ratio by town
LSTAT          - % lower status of the population
MEDV          - Median value of owner-occupied homes in $1000's

cols = ['RM', 'AGE', 'TAX', 'LSTAT', 'MEDV'] source: https://subscription.packtpub.com/book/programming/9781789804744/1/ch01lvl1sec11/our-first-analysis-the-boston-housing-dataset

df[cols].corr()

In [ ]:
#sns.pairplot(df[['MEDV', 'RM','LSTAT','AGE', 'PTRATIO', 'RAD', 'TAX']], hue='RAD') #create grid of plots where each variable is compared and colours determine the property type 
sns.pairplot(df[['MEDV', 'RM','LSTAT','AGE', 'PTRATIO', 'RAD', 'TAX']]) #create grid of plots where each variable is compared and colours determine the property type 

#Source:https://seaborn.pydata.org/generated/seaborn.pairplot.html

---------

Histograms 
MEDV and RM are closest to normal distribution - do somethign with MEDV normal distribution??https://subscription.packtpub.com/book/programming/9781789804744/1/ch01lvl1sec11/our-first-analysis-the-boston-housing-dataset

Odd 24 RAD is seen and tax at 700 (potentially capped) 

# Section X: Further Investigation LSTAT vs MDEV and RM vs MDEV 

### Trialing LSTAT & RM Lines for BestFit 

First lets use the numpy polyfit to find thebest fit striaght line for these data sets and visualise the results.

In [ ]:
x_dfRM = df['RM']
y_dfRM = df['MEDV']
plt.subplot(1, 2, 2)            #source:https://stackoverflow.com/questions/42818361/how-to-make-two-plots-side-by-side-using-python
plt.plot(x_dfRM,y_dfRM, 'bo',  label='LSTAT Original Data Points')   #Reference: Ian McLoughlin "Simple Linear Regression with NumPy" Jupyter Notebook, Semester 2 GMIT 
z_dfRM= np.polyfit(x_dfRM, y_dfRM, 1)
pRM = np.poly1d(z_dfRM) #use ployfit function to determine least squares polynomial line fit, where 1 is the Degree of the fitting the polynomial

plt.plot(x_dfRM,pRM(x_dfRM),"g--", label='PolyFit Best Straight Line RM') #Reference: Ian McLoughlin "Simple Linear Regression with NumPy" Jupyter Notebook, Semester 2 GMIT 
plt.xlabel('RM')
plt.ylabel('')
plt.legend()
#where the resuting m and c are values in the equation of a straight line (y=mx+c)
#pRM  # Y = 9.10210898 x + (-34.67062078)

############################################

x_df2 = df['LSTAT']
y_df2 = df['MEDV']

plt.subplot(1, 2, 1)
plt.plot(x_df2,y_df2, 'bo', label='LSTAT Original Data Points')   #Reference: Ian McLoughlin "Simple Linear Regression with NumPy" Jupyter Notebook, Semester 2 GMIT 
z_df2= np.polyfit(x_df2, y_df2, 1)
p2 = np.poly1d(z_df2) #use ployfit function to determine least squares polynomial line fit, where 1 is the Degree of the fitting the polynomial

plt.plot(x_df2,p2(x_df2),"g--", label='PolyFit Best Straight Line LSTAT') #Reference: Ian McLoughlin "Simple Linear Regression with NumPy" Jupyter Notebook, Semester 2 GMIT 

plt.xlabel('LSTAT')
plt.ylabel('MEDV')

#where the resuting m and c are values in the equation of a straight line (y=mx+c)
#p2  # Y = -0.95004935x + 34.55384088

#############################################

plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
#or in seaborn with 95% confidence intervals https://subscription.packtpub.com/book/programming/9781789804744/1/ch01lvl1sec11/our-first-analysis-the-boston-housing-dataset
fig, ax = plt.subplots(1, 2)
sns.regplot('RM', 'MEDV', df, ax=ax[0], ci=95, scatter_kws={'alpha': 0.4})
sns.regplot('LSTAT', 'MEDV', df, ax=ax[1], ci=95, scatter_kws={'alpha': 0.4})

"The standard error of the mean, also called the standard deviation of the mean, is a method used to estimate the standard deviation of a sampling distribution." Source: https://explorable.com/standard-error-of-the-mean

![alt text](https://explorable.com/images/standard-error-of-the-mean.jpg "Title")

* σM = standard error of the mean

* σ = the standard deviation of the original distribution

* N = the sample size



It gives us a quantifiable method to determine how good a fit our "predictive", least squares method lines are. 

In [ ]:
from sklearn.metrics import r2_score

#function to calculate the performance score between true and predicted values based on the metric chosen. Source: https://www.ritchieng.com/machine-learning-project-boston-home-prices/
def performance_metric(y_true, y_predict):
    score = r2_score(y_true, y_predict)
    return score

#Calculate the performance of this model  
score = performance_metric(y_df2, p2(x_df2))

#Calculate the performance of this model  
score2 = performance_metric(y_dfRM, pRM(x_dfRM))

##############################

from sklearn.metrics import mean_squared_error #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html

def get_mse(df, feature, y, y_pred, target='MEDV'):
    # Get x, y to model
    y = df[target].values
    x = df[feature].values.reshape(-1,1)

    error = mean_squared_error(y, y_pred)
    print('Mean Squared Error of', feature, ' = {:.2f}'.format(error)) 

print ("LSTAT polyfit line has a coefficient of determination, R^2, of {:.3f}.".format(score), "or", round(score*100, 2) , "% accuracy")
print ("and... ")
get_mse(df, 'LSTAT', x_df2, (p2(x_df2)))
print (" ")    
print ("RM polyfit line has a coefficient of determination, R^2, of {:.3f}.".format(score2), "or", round(score2*100, 2) , "% accuracy")
print ("and... ")
get_mse(df, 'RM', x_dfRM, (pRM(x_dfRM)))


LSTAT has a higher accuracy and lower error and so is the data I shall progress with.

However, when you look more closely to the LSTAT data, you notice that it in fact could get a better fit ( and lower Mean Squared Error) if using a curve rather than a straight line. Keeping in mind our observation that the MEDV data most likely was capped at $50k due to the number at this figure, applying a curved line also seems worth a try.

I want to try that before I begin modeling with the neural network..

In [ ]:
from scipy.optimize import curve_fit

def func(x, a, b, c):    #Source: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
    return a * np.exp(-b * x) + c     #Still uses least squares principle but is non linear

xdata = df['LSTAT']
ydata = df['MEDV']
plt.plot(xdata, ydata, 'ko', label='data')

#Fit for the parameters a, b, c of the function func:

popt, pcov = curve_fit(func, xdata, ydata)
popt

plt.plot(xdata, func(xdata, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

curve_1_y_data = func(xdata, *popt)

MSE1_curve1 = get_mse(df, 'LSTAT', xdata, func(xdata, *popt))

#Constrain the optimization to the region of 0 <= a <= 50, 0 <= b <= 1 and 0 <= c <= 0.5:

popt, pcov = curve_fit(func, xdata, ydata, bounds=(0, [55., 1., 1.0]))
popt

plt.plot(xdata, func(xdata, *popt), 'g--',
         label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
curve_2_y_data = func(xdata, *popt)

MSE1_curve2 = get_mse(df, 'LSTAT', xdata, func(xdata, *popt))

plt.xlabel('LSTAT')
plt.ylabel('MEDV')
plt.legend()
plt.show()

Sure enough the MSE reduces to approx. 28 with the red curve so really this is really worth keeping in mind as we begin to model with the neural network. 

SIDE-NOTE: this makes sense as when doing some training with Keras (where the LOSS is selected as MEAN sQUARED ERROR), BEFORE normalisation, I could only get down to a loss of about 27 (26.x, occasionally) for the LSTAT data. I was graphing the output each time and could see that the model was getting less error with "elu"/"relu" activations which were allowing more curvature in the predictive "line".

# Keras


In [ ]:
scaler = pre.StandardScaler().fit(df) #source:https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler.fit(df)

In [ ]:
df_scaled = pd.DataFrame(scaler.transform(df))
df_scaled.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
#df_scaled.sort_index() # = df_scaled1.sort_values(by=['PTRATIO'])

xscale_train_data = df_scaled.iloc[0:404, 0:13] 
noscale_train_targets = df.iloc[0:404, 13:14] 
xscale_test_data = df_scaled.iloc[404:507, 0:13]
noscale_test_targets = df.iloc[404:507, 13:14]
print(xscale_train_data.shape, xscale_train_targets.shape, xscale_test_data.shape, xscale_test_targets.shape)

=============================

In [ ]:
#inputs = preprocessing.scale(df[['LSTAT']]).ravel() #se np.ravel (for a 1D view), https://stackoverflow.com/questions/13730468/from-nd-to-1d-arrays
#output = preprocessing.scale(df['MEDV'])

inputs = xscale_train_data['LSTAT'].ravel()
output = noscale_train_targets['MEDV'].ravel()

In [ ]:
### Create a new neural network.
n = kr.models.Sequential()

# Add neurons
n.add(kr.layers.Dense(65, input_dim=1, activation="linear")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(100, activation="tanh")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(70, activation="sigmoid")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(45, activation="elu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(45, activation="selu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(10, activation="softmax")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(60, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.closen.add(kr.layers.Dense(45, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(60, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.closen.add(kr.layers.Dense(45, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(60, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.closen.add(kr.layers.Dense(45, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
n.add(kr.layers.Dense(1, activation="linear")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close

# Compile the model.
n.compile(loss="mean_squared_error", optimizer="Adadelta") # through testing the optomisers from the https://keras.io/optimizers/ website, the best was found to be Adadelta

# Train the model.
n.fit(inputs, output, epochs=25,batch_size=10)

# Run each x value through the neural network.
p_2 = n.predict(inputs)

In [ ]:
#p_2.T #estimated MEDV values using model
#output.as_matrix() #actual MEDV values

#calculate mean squared error
#np.sqrt(np.sum((p_2.T - output.as_matrix())**2)) 

In [ ]:
# Plot the values. #AdaDelta
plt.plot(inputs, output, 'co', label='Original Data Points')
#x_curve = np.linspace(4,9,100)
#y_curve = x_curve**2 + 9.10210898*r13 -34.67062078
#plt.plot(r13,t13, 'b-', label='Regression on Original Data Points')

fit = np.poly1d(np.polyfit(inputs, output, 1))
plt.plot(inputs, fit(inputs),'k-', label='Numpy Bestfit Straight Line')

plt.plot(inputs, p_2, 'r-', label='Prediction')
plt.legend()

In [ ]:
#Predict:
i=0
results=[]

while i<len(xscale_test_data):
    q_5 = n.predict([xscale_test_data['LSTAT'].iloc[i]])
#    print(q_5.type)
    results.append([q_5])
    i=i+1
    
#print(results)

In [ ]:
#### NOTE: can only run this cell once...
#if error flagged, just create results array again by running the above cell

results = pd.DataFrame(results)
results.iloc[:,0] = results.iloc[:,0].str.get(0) # needs to be used twice to eliminate brackets from DF however will not work in IF/WHILE loop!
results.iloc[:,0] = results.iloc[:,0].str.get(0)

results.columns=['Xscaled_Results']
results1 = results.values.ravel()

In [ ]:
# xscale_test_targets  =  the expected results we know from 102 entries in the original dataset
# LSTAT Estimation  =  the estimated results the neural network has produced

pd.options.display.max_rows = None
xscale_test_targets1 = xscale_test_targets
xscale_test_targets1['MEDV Estimation'] = results1
#xscale_test_targets['difference at scaled level'] = (xscale_test_targets['MEDV'])**2 - (xscale_test_targets['LSTAT Estimation'])**2
round(xscale_test_targets1,2)



In [ ]:
n.evaluate(xscale_test_targets1['MEDV Estimation'],xscale_test_targets1['MEDV'])

In [ ]:
#Predict:
test_set1 = pd.DataFrame([-1, -0.5, 0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5])
test_set = test_set1.as_matrix()
i=0
test_results=[]

while i<len(test_set):
    q_5 = n.predict([test_set[i]])
#    print(q_5.type)
    test_results.append([q_5, 0])
    i=i+1

test_results2 = pd.DataFrame(test_results)
test_results2.iloc[:,0] = test_results2.iloc[:,0].str.get(0)
test_results2.iloc[:,0] = test_results2.iloc[:,0].str.get(0)
test_results2["inputs"] = test_set
test_results2.drop([1], axis=1)
test_results2

In [ ]:
plt.plot(test_results2[0], test_results2['inputs'])

========================

In [ ]:
fit = np.poly1d(np.polyfit(xdata, ydata, 5))
plt.plot(xdata, fit(xdata),'k-', label='Numpy Bestfit LSTAT')
z_df= np.polyfit(inputs2, output2, 1)
p3 = np.poly1d(z_df) #use ployfit function to determine least squares polynomial line fit, where 1 is the Degree of the fitting the polynomial
plt.plot(inputs2,p3(inputs2),"g--", label='Numpy Bestfit PTRATIO') #Reference: Ian McLoughlin "Simple Linear Regression with NumPy" Jupyter Notebook, Semester 2 GMIT 

plt.show()
plt.legend()

In [ ]:
#df_copy3 = df[['MEDV', 'PTRATIO', 'LSTAT']].copy()

In [ ]:
inputs2 = xscale_train_data['PTRATIO'].ravel()
output2 = noscale_train_targets['MEDV'].ravel()

In [ ]:
# Create a new neural network.
o = kr.models.Sequential()

# Add neurons
o.add(kr.layers.Dense(50, input_dim=1, activation="linear")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
o.add(kr.layers.Dense(20, activation="linear")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
o.add(kr.layers.Dense(1, activation="linear")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close


# Compile the model.
o.compile(loss="mean_squared_error", optimizer="Adadelta") # through testing the optomisers from the https://keras.io/optimizers/ website, the best was found to be Adadelta

# Train the model.
o.fit(inputs2, output2, epochs=20,batch_size=10)

# Run each x value through the neural network.
p_3 = o.predict(inputs2)

In [ ]:
# Plot the values. #AdaDelta
#plt.plot(inputs, output, 'co', label='Original Data Points')
plt.plot(inputs2,output2, 'bo', label='Original Data')
plt.plot(inputs2, p_3, 'r-', label='Prediction')

z_df2= np.polyfit(inputs2, output2, 1)
p3 = np.poly1d(z_df2) #use ployfit function to determine least squares polynomial line fit, where 1 is the Degree of the fitting the polynomial
plt.plot(inputs2,p3(inputs2),"g--", label='Numpy Bestfit') #Reference: Ian McLoughlin "Simple Linear Regression with NumPy" Jupyter Notebook, Semester 2 GMIT 
plt.legend()
plt.show()

#where the resuting m and c are values in the equation of a straight line (y=mx+c)
p3  # Y = -4.00198154 x + 23.38138783

In [ ]:

#function to calculate the performance score between true and predicted values based on the metric chosen. Source: https://www.ritchieng.com/machine-learning-project-boston-home-prices/
def performance_metric(y_true, y_predict):
    score = r2_score(y_true, y_predict)
    return score

#Calculate the performance of this model  
score = performance_metric(output2, p3(inputs2))
print ("Line (green) has a coefficient of determination, R^2, of {:.3f}.".format(score), "or", round(score*100, 2) , "% accuracy")

============================##############################=======================================

In [ ]:
inputs3 = xscale_train_data[['PTRATIO', 'LSTAT']]
output3 = noscale_train_targets['MEDV'].ravel()

In [ ]:
### Create a new neural network.
q = kr.models.Sequential()

# Add neurons
q.add(kr.layers.Dense(20, input_dim=2, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
q.add(kr.layers.Dense(45, activation="linear")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
q.add(kr.layers.Dense(60, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
q.add(kr.layers.Dense(60, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
q.add(kr.layers.Dense(20, activation="relu")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
q.add(kr.layers.Dense(1, activation="linear")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close

# Compile the model.
q.compile(loss="mean_squared_error", optimizer="Adadelta") # through testing the optomisers from the https://keras.io/optimizers/ website, the best was found to be Adadelta

# Train the model.
q.fit(inputs3, output3, epochs=2000,batch_size=10)

# Run each x value through the neural network.
q_3 = q.predict(inputs3)

In [ ]:
n.evaluate(Results_df_2['Estimation Results1'],Results_df_2['MEDV'])

In [ ]:
#Calculate the performance of this model but only if viewing it as a linear model 
#this metric should be taken with a pinch of salt as although both are similar we have seen LSTAT likes to curve a lot more than the model for PTRATIO
score = performance_metric(output3, q_3)
print ("Line (green) has a coefficient of determination, R^2, of {:.3f}.".format(score), "or", round(score*100, 2) , "% accuracy")

In [ ]:
#Predict:
i=0
results2=[]

while i<len(xscale_test_data['LSTAT']):
    q_5 = q.predict(pd.DataFrame(xscale_test_data[['PTRATIO', 'LSTAT']].iloc[i]).T)
#    print(q_5.type)
    results2.append([q_5,0])
    i=i+1
    
#print(results2)

In [ ]:
#### NOTE: can only run this cell once...
#if error flagged, just create results array again by running the above cell

results2 = pd.DataFrame(results2)
results2.iloc[:,0] = results2.iloc[:,0].str.get(0) # needs to be used twice to eliminate brackets from DF however will not work in IF/WHILE loop!
results2.iloc[:,0] = results2.iloc[:,0].str.get(0)

results2.columns=['Xscaled_Results', 'Extra']
results2.drop(columns=['Extra'])
#results2a = results2.values.ravel()
#results2b = pd.DataFrame(results2a)#.drop(columns=['Extra']) 
results2b = pd.DataFrame(results2).iloc[0:102, 0]

In [ ]:
# xscale_test_targets  =  the expected results we know from 102 entries in the original dataset
# LSTAT Estimation  =  the estimated results the neural network has produced

pd.options.display.max_rows = None
xscale_test_targets.drop(columns=['MEDV Estimation'], axis=1)
Results_df_2 = xscale_test_targets
Results_df_2['Estimation Results1'] = results2b.values.ravel()

#xscale_test_targets['difference at scaled level'] = (xscale_test_targets['MEDV'])**2 - (xscale_test_targets['LSTAT Estimation'])**2
round(Results_df_2,2)

In [ ]:
plt.plot(inputs3['PTRATIO'], output3, 'go')
plt.plot(inputs3['LSTAT'], output3, 'yo')

q3 = np.poly1d(np.polyfit(inputs3['PTRATIO'], output2, 1)) #use ployfit function to determine least squares polynomial line fit, where 1 is the Degree of the fitting the polynomial
plt.plot(inputs3['PTRATIO'],q3(inputs3['PTRATIO']),"r-", label='Numpy Bestfit PTRATIO') #Reference: Ian McLoughlin "Simple Linear Regression with NumPy" Jupyter Notebook, Semester 2 GMIT 

q3a = np.poly1d(np.polyfit(inputs3['LSTAT'], output3, 5))
plt.plot(inputs3['LSTAT'], q3a(inputs3['LSTAT']),'k-', label='Numpy Bestfit LSTAT')

#plt.plot(xscale_test_data, noscale_test_targets, 'ro', marker='.')

#plt.plot(pd.DataFrame(scaler.transform(Results_df_2['Estimation Results1'])), noscale_test_targets, 'bo', marker='.')


plt.plot(inputs3, q_3, 'co', label='Prediction')


plt.legend()
plt.show()


In [ ]:
input_test = [[18,2.5],[15,2],[35,6]] 
q_5a = pd.DataFrame(input_test)
q_5 = q.predict(q_5a)
q_5

--------

### What if the size of the property is a factor to consider?
MEDV is a curious feature as it appears to be a total value for the property not taking into account for the size of it. After location, the size or quality of the building are surely the biggest factors influencing price. 

As we do have a column that is an inidicator of size, the average number of rooms per dwelling (RM), I will divide the total price by the average number of rooms to get an indicative square metre price. 

In [ ]:
df_sm = df.filter(items=['CHAS', 'RM','MEDV'])
df_sm2 =  df_sm.MEDV / df_sm.RM 
df_sm3 = pd.concat([df_sm, df_sm2], axis=1, sort=False).drop(['RM', 'MEDV'], axis=1)
df_sm3.columns = ('CHAS', 'SME') #SME for Square Metre estimate
#df_sm3

sme_t = (df_sm3[df_sm3['CHAS'] == 0].drop(['CHAS'], axis=1))
sme_f = (df_sm3[df_sm3['CHAS'] == 1].drop(['CHAS'], axis=1))


In [ ]:
ss.ttest_ind(sme_t, sme_f)

P value is still not close enough 

In [ ]:
sns.distplot(sme_t, color='k', axlabel='MEDV') # house MEDV prices close to the river are in b;ack
sns.distplot(sme_f, color='g') # house MEDV prices farther from the river are in green

====

# Extra investigation into RAD and TAX relationship

-----------------------------

Firstly I built a function to split out data and allow me to review for trends before deciding which characteristics to choose for predictions.

The function is based on the below principles:

That the unique groupigns can be identified easily using .unique(),
`df1.RAD.unique()` 


and each group then separated for further analysis.
`df_rad1 = ((df1[df1['RAD'] == "4"]))`

The first review was for RAD vs TAX based on the correlation we could see in the heat map above. 

In [ ]:
def impact(rating, column_name_string1, column_name_string2, column_name_string_return):
        df_rad1 = ((df1[df1[column_name_string1] == str(rating)]))
        df_rad11 = pd.concat([df[column_name_string2], df_rad1], axis=1, sort=False)
        rad_1y = ((df_rad11[df_rad11[column_name_string1] == True]).drop([column_name_string1], axis=1))
        rad_1y1 = rad_1y.values
        #len_1x = len(rad_1y)
        rad_1x = []
        i = 0
        while i<len(rad_1y):
            i=i+1
            rad_1x.append(rating)
        return df_rad1[column_name_string_return]
    
#####################################

run1 = impact(1, 'RAD', 'TAX', 'TAX')
run2 = impact(2, 'RAD', 'TAX', 'TAX')
run3 = impact(3, 'RAD', 'TAX', 'TAX')
run4 = impact(4, 'RAD', 'TAX', 'TAX')
run5 = impact(5, 'RAD', 'TAX', 'TAX')
run6 = impact(6, 'RAD', 'TAX', 'TAX')
run7 = impact(7, 'RAD', 'TAX', 'TAX')
run8 = impact(8, 'RAD', 'TAX', 'TAX')
run9 = impact(24, 'RAD', 'TAX', 'TAX')

#Check point:
#print(len(rad_impact(24))+len(rad_impact(1))+ len(rad_impact(2))+ len(rad_impact(3))+ len(rad_impact(4))+ len(rad_impact(5))+ len(rad_impact(6))+ len(rad_impact(7))+ len(rad_impact(8)))

radx = pd.concat([run1, run2, run3, run4, run5, run6, run7, run8, run9], axis=1)
radx.columns=("1", "2","3" ,"4" ,"5" ,"6" ,"7" ,"8", "24")

run10 = impact(1, 'RAD', 'TAX', 'MEDV')
run11 = impact(2, 'RAD', 'TAX', 'MEDV')
run12 = impact(3, 'RAD', 'TAX', 'MEDV')
run13 = impact(4, 'RAD', 'TAX', 'MEDV')
run14 = impact(5, 'RAD', 'TAX', 'MEDV')
run15 = impact(6, 'RAD', 'TAX', 'MEDV')
run16 = impact(7, 'RAD', 'TAX', 'MEDV')
run17 = impact(8, 'RAD', 'TAX', 'MEDV')
run18 = impact(24, 'RAD', 'TAX', 'MEDV')

#Check point:
#print(len(rad_impact(24))+len(rad_impact(1))+ len(rad_impact(2))+ len(rad_impact(3))+ len(rad_impact(4))+ len(rad_impact(5))+ len(rad_impact(6))+ len(rad_impact(7))+ len(rad_impact(8)))

radx_2 = pd.concat([run10, run11, run12, run13, run14, run15, run16, run17, run18], axis=1)
radx_2.columns=("1", "2","3" ,"4" ,"5" ,"6" ,"7" ,"8", "24")

In [ ]:
ax = sns.catplot(kind="box",data=radx, showmeans=True)
ax.set(xlabel='RAD Rating', ylabel='TAX $')

ay = sns.catplot(kind="box",data=radx_2, showmeans=True)
ay.set(xlabel='RAD Rating', ylabel='MEDV $')


In [ ]:
radx1 = radx.astype(float)
radx2 = radx1.mean(axis=0, skipna=True)
rad_mean = pd.DataFrame(radx2).T
rad_mean.columns=("1 (mean)", "2 (mean)","3 (mean)" ,"4 (mean)" ,"5 (mean)" ,"6 (mean)" ,"7 (mean)" ,"8 (mean)", "24 (mean)")
rx1 = ["1 (mean TAX)", "2 (mean)","3 (mean)" ,"4 (mean)" ,"5 (mean)" ,"6 (mean)" ,"7 (mean)" ,"8 (mean)", "24 (mean)"]
ry1 = rad_mean.iloc[0,:]
round(rad_mean, 3)

In [ ]:
radx1 = radx_2.astype(float)
radx2 = radx1.mean(axis=0, skipna=True)
rad_mean = pd.DataFrame(radx2).T
rad_mean.columns=("1 (mean)", "2 (mean)","3 (mean)" ,"4 (mean)" ,"5 (mean)" ,"6 (mean)" ,"7 (mean)" ,"8 (mean)", "24 (mean)")
rx2 = ["1 (mean MEDV)", "2 (mean)","3 (mean)" ,"4 (mean)" ,"5 (mean)" ,"6 (mean)" ,"7 (mean)" ,"8 (mean)", "24 (mean)"]
ry2 = rad_mean.iloc[0,:]
round(rad_mean, 3)

In [ ]:
plt.plot(rx1, ry1, '-bo')

In [ ]:
plt.plot(rx2, ry2, '-bo')

In [ ]:
sns.pairplot(df_rad24[['MEDV', 'RM','LSTAT', 'RAD', 'CRIM', 'NOX', 'AGE']], hue='MEDV') #create grid of plots where each variable is compared and colours determine the property type 

#Source:https://seaborn.pydata.org/generated/seaborn.pairplot.html

In [ ]:
df_rad24 = ((df[df['RAD'] == 24]))  
df_rad24

In [ ]:
run9 = impact(24, 'RAD', 'TAX', 'TAX')

run18 = impact(24, 'RAD', 'TAX', 'MEDV')

run19 = impact(24, 'RAD', 'CRIM', 'MEDV')

radx_2 = pd.concat([run9, run18], axis=1)


CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT MEDV


## End

--------------------------------

# Back up

# Deciding if LSTATS or RM should be used - but not both!

Test the most obvious correlation - rooms (indication of size) vs price 

In [ ]:
df_copy1 = df[['MEDV', 'RM']].copy()
df_copy2 = df_copy1.sort_values(by=['RM'])

In [ ]:
x_df = df_copy2['RM']
y_df = df_copy2['MEDV']
plt.plot(x_df,y_df, 'bo')
z_df= np.polyfit(x_df, y_df, 1)
p = np.poly1d(z_df) #use ployfit function to determine least squares polynomial line fit, where 1 is the Degree of the fitting the polynomial

plt.plot(x_df,p(x_df),"g--") #Reference: Ian McLoughlin "Simple Linear Regression with NumPy" Jupyter Notebook, Semester 2 GMIT 
plt.show()
#where the resuting m and c are values in the equation of a straight line (y=mx+c)
p  # Y = 9.10210898 x + (-34.67062078)

In [ ]:
from sklearn.metrics import r2_score

#function to calculate the performance score between true and predicted values based on the metric chosen. Source: https://www.ritchieng.com/machine-learning-project-boston-home-prices/
def performance_metric(y_true, y_predict):
    score = r2_score(y_true, y_predict)
    return score

#Calculate the performance of this model  
score = performance_metric(y_df, p2(x_df))
print ("Line (green) has a coefficient of determination, R^2, of {:.3f}.".format(score), "or", round(score*100, 2) , "% accuracy")

==========Modeling RM=========

In [ ]:
#x = np.linspace(-10.0, 10.0, 506)
x = df_copy2 ['RM']
y = df_copy2 ['MEDV']


# Create a new neural network.
m = kr.models.Sequential()

# Add neurons.
#m.add(kr.layers.Dense(1, input_dim=1, activation="linear"))

# Add neurons
m.add(kr.layers.Dense(40, input_dim=1, activation="tanh")) #adjusting from 10 to 30,improves loss to 41.x at epoch 45, visually looks v.close
m.add(kr.layers.Dense(60, activation="tanh"))
m.add(kr.layers.Dense(1, activation='linear'))

# Compile the model.
m.compile(loss="mean_squared_error", optimizer="Adadelta") # through testing the optomisers from the https://keras.io/optimizers/ website, the best was found to be Adadelta
# got to 42,x during testing with 60epochs, no change to neuron numbers. 

# Train the model.
m.fit(x, y, epochs=60,batch_size=10)

# Run each x value through the neural network.
p = m.predict(x)

In [ ]:
# Plot the values. #AdaDelta
plt.plot(x, p, 'r-', label='Prediction')
plt.plot(x, y, 'ko', label='Original Data Points')
r13 = np.linspace(4,9,100)
t13 = 9.10210898*r13 -34.67062078
plt.plot(r13,t13, 'b-', label='Regression on Original Data Points')
plt.legend()

==============

In [ ]:
#or in seaborn with 95% confidence intervals https://subscription.packtpub.com/book/programming/9781789804744/1/ch01lvl1sec11/our-first-analysis-the-boston-housing-dataset
fig, ax = plt.subplots(1, 2)
ax[0] = sns.residplot('RM', 'MEDV', df, ax=ax[0],
                      scatter_kws={'alpha': 0.4})
ax[0].set_ylabel('MDEV residuals $(y-\hat{y})$')
ax[1] = sns.residplot('LSTAT', 'MEDV', df, ax=ax[1],
                      scatter_kws={'alpha': 0.4})
ax[1].set_ylabel('')

Each point on these residual plots is the difference between that sample (y) and the linear model prediction ( ŷ). Residuals greater than zero are data points that would be underestimated by the model. Likewise, residuals less than zero are data points that would be overestimated by the model.

Patterns in these plots can indicate sub optimal modeling. In each preceding case,we see diagonally arranged scatter points in the positive region. These are caused by the $50,000 cap on MEDV. The RM data is clustered nicely around 0, which indicates a good fit. On the other hand, LSTAT appears to be clustered lower than 0.

 